## Task 1 - Virtual Environment

#### 1.1 Change to a Free GPU Runtime

In [0]:
# TASK 1.1: Check if you are currently using the GPU in Colab

import tensorflow as tf
tf.test.gpu_device_name()

#### TASK 1.2: Mount Google Drive

In [0]:
# Task: 1.2.1 Install google-drive-ocamlfuse

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
# Task: 1.2.2 Authenticate and get credentials

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# TASK 1.2.3: Mount Google Drive in local Colab VM

!mkdir -p drive
!google-drive-ocamlfuse drive
!ls

In [0]:
!ls drive

In [0]:
!ls drive/NTUOSS-ImageRecognitionWorkshop

In [0]:
!ls drive/NTUOSS-ImageRecognitionWorkshop/data

## Task 2 - Preprocess Images

#### 2.1 Configure Image Augmentation

In [0]:
# TASK 2.1 : Add augmentation configuration for the data generator of train data only

from keras.preprocessing.image import ImageDataGenerator

datagen_train =  ImageDataGenerator(
    rescale = 1. / 255,
    rotation_range = 30,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)
datagen_val = ImageDataGenerator(
    rescale = 1. / 255)

#### 2.2 Generate Image Data from Directory

In [0]:
# TASK 2.2.1 : Generate Image Data from Directory and Set parameter

train_data = datagen_train.flow_from_directory(
    directory = './drive/NTUOSS-ImageRecognitionWorkshop/data/train',
    target_size = (150, 150),
    class_mode = 'binary',
    shuffle = True,
    batch_size = 50)
validation_data = datagen_val.flow_from_directory(
    directory = './drive/NTUOSS-ImageRecognitionWorkshop/data/validation',
    target_size = (150, 150),
    class_mode = 'binary',
    shuffle = True,
    batch_size = 50)

In [0]:
# TASK 2.2.2 : Check class indices
print('Class Indices : {}'.format(train_data.class_indices))

## Task 3 - Build Basic Model

#### 3.1 Set up backend and Import libraries

In [0]:
# TASK 3.1.1 Configure backend

from keras import backend as K
K.set_image_dim_ordering('tf') #channel last
K.set_image_data_format('channels_last')

In [0]:
# TASK 3.1.2 Import the Keras libraries and packages

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

#### 3.2 Construct Model

In [0]:
# TASK 3.2.1 Initialize Neural Network Model
model = Sequential()

# TASK 3.2.2 Create first set of CONV -> RELU -> POOL layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3))) # output shape = (148, 148, 32)
model.add(MaxPooling2D(pool_size=(2, 2))) # output shape = (74, 74, 32)

# TASK 3.2.3 Create second set of CONV -> RELU -> POOL layers
model.add(Conv2D(64, (3, 3), activation='relu')) # output shape = (72, 72, 64)
model.add(MaxPooling2D(pool_size=(2, 2))) # output shape = (36, 36, 64)

# TASK 3.2.4 Create third set of CONV -> RELU -> POOL layers
model.add(Conv2D(64, (3, 3), activation='relu')) # output shape = (34, 34, 64)
model.add(MaxPooling2D(pool_size=(2, 2))) # output shape = (17, 17, 64)

# TASK 3.2.5 Convert the 3D feature maps to 1D feature vectors
model.add(Flatten()) # output shape = (18496,)

# TASK 3.2.6 Add the connection layer
model.add(Dense(units = 256, activation='relu')) # output shape = (256,)

# TASK 3.2.7 Add the output layer
model.add(Dense(units = 1, activation = 'sigmoid')) # output shape = (1,)

# TASK 3.2.8 Compile the model
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

#### 3.3 Check Model

In [0]:
# TASK 3.3 Check the model structure.
model.summary()

## Task 4 - Train Model



In [0]:
# TASK 4: Train Model [WARNING: It took me 84.95 min]
import time
train_start_time = time.time()

from keras.callbacks import EarlyStopping
model.fit_generator(
    generator = train_data,
    epochs = 25,
    callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0)],
    validation_data = validation_data)
model.save('./drive/NTUOSS-ImageRecognitionWorkshop/cnn_model_basic.h5') # save model
print("It takes {:.2f} min to train the model".format((time.time() - train_start_time)/60 ))

## Task 5 - Test Basic Model

#### 5.1 Load Model

In [0]:
# Skip this if you just finished training the model and the model exists in the working environment

# TASK 5.1: Load trained model
from keras.models import load_model
model_basic = load_model('./drive/NTUOSS-ImageRecognitionWorkshop/data/model/cnn_model_basic.h5')
# model_basic = load_model('./drive/NTUOSS-ImageRecognitionWorkshop/cnn_model_basic.h5') # To load the model trained by you

#### 5.2 Generate Test Image Data from Directory

In [0]:
# TASK 5.2.1: Set up data generator for test data
from keras.preprocessing.image import ImageDataGenerator
datagen_test = ImageDataGenerator(rescale=1. / 255)

test_data = datagen_test.flow_from_directory (
    directory = './drive/NTUOSS-ImageRecognitionWorkshop/data/test',
    target_size = (150, 150),
    class_mode = None,
    shuffle = False,
    batch_size = 200)


In [0]:
# TASK 5.2.2: Check test generator
print(test_data.class_indices)
print(test_data.filenames)

#### 5.3 Make Prediction

In [0]:
# TASK 5.3.1: Use model to yield probability prediction for test data
probabilities = model_basic.predict_generator(test_data)
print(probabilities)

In [0]:
# TASK 5.3.2: Process probabilities to get prediction result
y_pred = [1 if prob > 0.5 else 0 for prob in probabilities]
print(y_pred)

In [0]:
# TASK 5.3.3: Prepare actual result using filenames
y_true = [0 if 'cat' in filename else 1 for filename in test_data.filenames]
print(y_true)


In [0]:
# TASK 5.3.4: Calculate accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true, y_pred))

In [0]:
# TASK 5.3.5: Generate a report
import pandas as pd
pd.crosstab(pd.Series(y_true), pd.Series(y_pred), rownames = ['True'], colnames = ['Pred'], margins = True)

## Task 6 - Recognize Image with Basic Model

#### 6.1 Useful Functions

In [0]:
# TASK 6.1.1: Define a function for reading image from url
def read_image_from_url(url):
    try:
        import requests, io
        from PIL import Image
        r = requests.get(url, timeout=15)
        img = Image.open(io.BytesIO(r.content))
        return img
    except:
        print("{:<10} Cannot find image from {}".format('[ERROR]', url))
        exit(1)

In [0]:
# TASK 6.1.2: Define a function for preprocessing image
def preprocess_image(img, target_size):
    from PIL import Image
    import numpy as np
    from keras.preprocessing.image import img_to_array
    img = img.resize(target_size,Image.ANTIALIAS)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img.astype('float32')
    img /= 255
    return img


In [0]:
# TASK 6.1.3: Define a function for processing result
def process_result(prob):
    return 'dog' if prob > 0.5 else 'cat'

#### 6.2 Make Prediction

In [0]:
# TASK 6.2: Read image, Preprocess image, and Make prediction
image = read_image_from_url('https://www.readersdigest.ca/wp-content/uploads/2011/01/4-ways-cheer-up-depressed-cat.jpg') # replace with any image url
image = preprocess_image(image, (150, 150))
prob = model_basic.predict(image)
print('Probability: ' + str(prob))
print('Class: ' + process_result(prob))

## Task 7 - Build on Top of Pretrained Model

#### 7.1 Set up backend and Import libraries

In [0]:
# TASK 7.1.1 Configure backend
from keras import backend as K
K.set_image_dim_ordering('tf') #channel last
K.set_image_data_format('channels_last')

In [0]:
# TASK 7.1.2 Import the Keras libraries and packages
from keras.applications import VGG16
from keras.layers import Dense, Dropout, Flatten
from keras.models import Model

#### 7.2 Construct Model

In [0]:
# TASK 7.2.1: Load base model
base_model = VGG16(include_top = False, weights = 'imagenet', input_shape = (150, 150, 3))

In [0]:
# TASK 7.2.2: Add new layers
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x) #new FC layer, random init
x = Dropout(0.4)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [0]:
# TASK 7.2.3: Setup trainable layer
for layer in base_model.layers:
    layer.trainable = False
print("{:<10} Pretrained model layers: {}".format('[INFO]', len(base_model.layers)))
print("{:<10} Total number of layers : {}".format('[INFO]', len(model.layers)))


In [0]:
# TASK 7.2.4: Compile model
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

#### 7.3 Check Model

In [0]:
# TASK 7.3: Check Model
model.summary()

## Task 8 - Train Advanced Model

In [0]:
# TASK 8: Train advanced model.

import time
train_start_time = time.time()

from keras.callbacks import EarlyStopping

model.fit_generator(
    train_data,
    epochs= 25 ,
    callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0)],
    validation_data = validation_data)
model.save('./drive/NTUOSS-ImageRecognitionWorkshop/cnn_model_advanced.h5')

print("It takes {:.2f} min to train the model".format((time.time() - train_start_time)/60 ))

## Task 9 - Test Advanced Model

#### 9.1 Load Model

In [0]:
# Skip this if you just finished training the model and the model exists in the working environment

# TASK 5.1: Load trained model
from keras.models import load_model
model_advanced = load_model('./drive/NTUOSS-ImageRecognitionWorkshop/data/model/cnn_model_advanced.h5')
# model_basic = load_model('./drive/NTUOSS-ImageRecognitionWorkshop/cnn_model_advanced.h5') # To load the model trained by you

#### 9.2 Generate Test Image Data from Directory

In [0]:
# TASK 9.2.1 : Set up data generator for test data
from keras.preprocessing.image import ImageDataGenerator
datagen_test = ImageDataGenerator(rescale=1. / 255)

test_data = datagen_test.flow_from_directory (
    directory = './drive/NTUOSS-ImageRecognitionWorkshop/data/test',
    target_size = (150, 150),
    class_mode = None,
    shuffle = False,
    batch_size = 200)  

In [0]:
# TASK 9.2.2: Check test data generator
print(test_data.class_indices)
print(test_data.filenames)

#### 9.3 Make Prediction

In [0]:
# TASK 9.3.1: Use model to yield probability prediction for test data
probabilities = model_advanced.predict_generator(test_data)
print(probabilities)

In [0]:
# TASK 9.3.2: Process probabilities to get prediction result
y_pred = [1 if prob > 0.5 else 0 for prob in probabilities]
print(y_pred)

In [0]:
# TASK 9.3.3: Prepare actual result using folder name in filenames
y_true = [0 if 'cat' in filename else 1 for filename in test_data.filenames]
print(y_true)

In [0]:
# TASK 9.3.4: Calculate accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true, y_pred))

In [0]:
# TASK 9.3.5: Generate a report
import pandas as pd
pd.crosstab(pd.Series(y_true), pd.Series(y_pred), rownames = ['True'], colnames = ['Pred'], margins = True)

## Task 10 - Recognize Image with Advanced Model

#### 10.1 Make Prediction

In [0]:
# TASK 10.1: Read image, Preprocess image, and Make prediction
image = read_image_from_url('https://www.readersdigest.ca/wp-content/uploads/2011/01/4-ways-cheer-up-depressed-cat.jpg') # replace with any image url
image = preprocess_image(image, (150, 150))
prob = model_advanced.predict(image)
print('Probability: ' + str(prob))
print('Class: ' + process_result(prob))

In [0]:
# Restart Google Colab - Only run the line below when u mess up with the virtual machine
# !kill -9 -1